<a href="https://colab.research.google.com/github/hamidriasat/Natural-Language-Processing/blob/master/Incident%20Severity%20Scoring/Incident_Project_XLnet_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  To get additional memory/ram
# a = []
# while(1):
    # a.append("1")

In [ ]:
!pip install pytorch-transformers


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

from collections import Counter


Using TensorFlow backend.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
!gdown --id 1v21uOScs8OScfkldm_7KjpN4zj-S-VI6

!unzip OverAllDataset


In [ ]:

def readLabels():
  full_labels= []
  with open( "OverAllDataset/Labels.txt" , encoding="cp1252") as content_file:
    for line in content_file:
      full_labels.append( int(line))
  return full_labels

def readData( ):
    full_date= []
    for i in range(1, 1262):
        with open( "OverAllDataset/"+ str(i) + ".txt" , encoding="cp1252") as content_file:
            fileData= content_file.read()
            templength= len(fileData.split() )
            if templength < 300:
                full_date.append(fileData )
            else:
                temp= ""
                convert= fileData.split()
                for j in range(0,300):
                    temp= temp+ " "+convert[j]
                full_date.append(temp )

    return full_date


data_X= readData()
data_Y= readLabels()

print(len(data_X))
print(len(data_Y))

1261
1261


In [ ]:
for k in range(1255, 1261): 
  print(data_Y[k])


1
0
3
4
0
0


In [ ]:
sentences = data_X
labels = data_Y


In [ ]:
#  get number of occurrences for each element
Counter( data_Y )

Counter({0: 256, 1: 337, 2: 140, 3: 261, 4: 194, 5: 73})

In [ ]:
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]


In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased', do_lower_case=True)

100%|██████████| 798011/798011 [00:00<00:00, 5440140.31B/s]


In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁driver', '▁dead', '▁after', '▁crash', '▁with', '▁', 'georgia', '▁school', '▁bus', '▁carrying', '▁kids', '▁', 'g', 'win', 'nett', '▁county', ',', '▁', 'ga', '.', '▁', '-', '-', '▁police', '▁in', '▁', 'georgia', '▁say', '▁the', '▁driver', '▁of', '▁an', '▁', 's', 'uv', '▁has', '▁been', '▁killed', '▁in', '▁a', '▁crash', '▁with', '▁a', '▁bus', '▁carrying', '▁five', '▁elementary', '▁school', '▁students', '.', '▁in', '▁a', '▁statement', '▁posted', '▁to', '▁twitter', '▁by', '▁', 'g', 'win', 'nett', '▁police', ',', '▁officers', '▁confirmed', '▁none', '▁of', '▁the', '▁students', '▁were', '▁hurt', '.', '▁the', '▁bus', '▁driver', '▁suffered', '▁a', '▁minor', '▁injury', '▁but', '▁was', '▁not', '▁taken', '▁to', '▁a', '▁hospital', '.', '▁news', '▁outlets', '▁reported', '▁the', '▁', 's', 'uv', '▁crashed', '▁into', '▁the', '▁', 'g', 'win', 'nett', '▁county', '▁bus', '▁just', '▁after', '▁3', '▁p', '.', 'm', '.', '▁at', '▁an', '▁intersection', '▁about', '▁40', '▁miles', '▁

In [ ]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
MAX_LEN = 300

# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]


In [ ]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.2)

In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# Select a batch size for training. For fine-tuning with XLNet, 
# the authors recommend a batch size of 32, 48, or 128.
#  We will use 32 here to avoid memory issues.
batch_size = 16


# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=6)
# model = XLNetForSequenceClassification.from_pretrained("xlnet-large-cased", num_labels=6)

model.cuda()

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
 
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 1.7075177185119144


Epoch:  25%|██▌       | 1/4 [01:06<03:20, 66.80s/it]

Validation Accuracy: 0.23137019230769232
Train loss: 1.5682632052709187


Epoch:  50%|█████     | 2/4 [02:13<02:13, 66.65s/it]

Validation Accuracy: 0.4723557692307692
Train loss: 1.1746086724220761


Epoch:  75%|███████▌  | 3/4 [03:19<01:06, 66.54s/it]

Validation Accuracy: 0.6234975961538461
Train loss: 0.8492086718952845


Epoch: 100%|██████████| 4/4 [04:25<00:00, 66.42s/it]

Validation Accuracy: 0.6373197115384616


In [ ]:
# Let’s take a look at our training loss over all batches:

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()